In [1]:
import numpy as np
import graphinglib as gl
import pyregion
from scipy.optimize import curve_fit
import dill
from functools import partial

from src.statistics.advanced_stats import autocorrelation_function, autocorrelation_function_2d
from src.hdu.maps.map import Map

In [2]:
def get_plottables(data: np.ndarray, region_radius: float) -> list:
    sorted_data = data[np.argsort(data[:,0])]
    scatter = gl.Scatter(
        sorted_data[:,0],
        sorted_data[:,1],
        marker_size=2,
        face_color="orange",
    )

    scatter.add_errorbars(
        y_error=sorted_data[:,2],
        errorbars_line_width=0.25,
        cap_width=0,
    )

    return [scatter]

In [3]:
turbulence = Map.load("summer_2023/gaussian_fitting/maps/computed_data_selective/turbulence.fits")

regions = [
    ("Global region", None, 50),
    ("Diffuse region", pyregion.open("summer_2023/gaussian_fitting/regions/region_1.reg"), 20),
    ("Central region", pyregion.open("summer_2023/gaussian_fitting/regions/region_2.reg"), 10),
    ("Filament region", pyregion.open("summer_2023/gaussian_fitting/regions/region_3.reg"), 6)
]

### Autocorrelation functions of all regions

In [4]:
figs = []

for name, region, region_radius in regions:
    # if not region:
    #     try:
    #         with open("applications/sh158/saved_data/acr_func_turbulence_global.gz", "rb") as f:
    #             data = dill.load(f)
    #     except:
    #         data = autocorrelation_function(turbulence.get_masked_region(region).data)
    #         with open("applications/sh158/saved_data/acr_func_turbulence_global.gz", "wb") as f:
    #             dill.dump(data, f)
    # else:
    #     data = autocorrelation_function(turbulence.get_masked_region(region).data)
    data = autocorrelation_function(turbulence.get_masked_region(region).data)

    fig = gl.Figure(
        title=name,
        x_label="Lag",
        y_label="Autocorrelation function",
        x_lim=(0, region_radius*1.2),
    )
    plottables = get_plottables(data, region_radius)
    cropped_scatter = plottables[0].create_slice_x(0, region_radius*1.2)
    fig.y_lim = np.min(cropped_scatter.y_data)-0.2, np.max(cropped_scatter.y_data)+0.3
    fig.add_elements(*plottables)
    figs.append(fig)

multifig = gl.MultiFigure.from_grid(figs, (2,2), (14, 9))
multifig.save("figures/sh158/advanced_stats/autocorrelation/autocorrelation_functions_new.pdf")

### 2D Autocorrelation function

In [5]:
# print("STARTING")
# try:
#     print("Here 1")
#     with open("applications/sh158/saved_data/2d_acr_func_turbulence_global.gz", "rb") as f:
#         data = dill.load(f)
# except:
#     print("Here 2")
#     data = autocorrelation_function_2d(turbulence.data)
#     with open("applications/sh158/saved_data/2d_acr_func_turbulence_global.gz", "wb") as f:
#         dill.dump(data, f)
data = autocorrelation_function_2d(turbulence.data)

fig = gl.Figure()
scat = gl.Scatter(
    x_data=data[:,0],
    y_data=data[:,1],
    face_color=data[:,2],
    show_color_bar=True,
    marker_size=2,
)
fig.add_elements(scat)
fig.save("figures/sh158/advanced_stats/autocorrelation/autocorrelation_function_2d_new.pdf")